# **Pipeline para usuarios no logueados**

1. **Cargar y procesar el dataset de prueba**:
   - Identificar sesiones donde `user_id = -1`.

2. **Generar recomendaciones**:
   - Basado en popularidad global (familias y colores más populares).
   - Ajustar por atributos contextuales (`pagetype`, `device_type`).

3. **Formato de predicciones**:
   - Generar el archivo JSON con 5 recomendaciones únicas por `session_id`.

In [5]:
import pandas as pd
import json

# Cargar datasets de prueba y productos
test_df = pd.read_pickle("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl")
products_df = pd.read_pickle("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data_enriched.pkl")

# Familias y colores más populares (globales)
popular_families = products_df['family'].value_counts().index.tolist()
popular_colors = products_df['color_id'].value_counts().index.tolist()

# Filtrar valores negativos en 'partnumber'
products_df = products_df[products_df['partnumber'] > 0]

# Generar recomendaciones para usuarios no logueados
def generate_unlogged_recommendations(test_df, products_df):
    """Genera recomendaciones para usuarios no logueados en el formato correcto."""
    predictions = {}
    unlogged_sessions = test_df[test_df['user_id'] == -1]

    for session_id in unlogged_sessions['session_id'].unique():
        # Selección de productos (basado en popularidad)
        recommended_products = products_df[
            (products_df['family'].isin(popular_families)) &
            (products_df['color_id'].isin(popular_colors))
        ]['partnumber'].drop_duplicates().head(5).tolist()

        # Asegurar valores positivos y agregar al diccionario de predicciones
        predictions[str(session_id)] = [int(product) for product in recommended_products]

    return {"target": predictions}

# Generar predicciones
unlogged_predictions = generate_unlogged_recommendations(test_df, products_df)

# Guardar predicciones en formato JSON
output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_unlogged.json"
with open(output_path, 'w') as f:
    json.dump(unlogged_predictions, f, indent=4)

print(f"Predicciones para usuarios no logueados guardadas en {output_path}")



Predicciones para usuarios no logueados guardadas en /home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_unlogged.json


In [9]:
# Inspeccionar valores únicos en 'partnumber'
print("Valores únicos en 'partnumber':")
print(products_df['partnumber'].value_counts())

# Verificar la frecuencia de -1
null_partnumber_count = (products_df['partnumber'] == -1).sum()
print(f"\nCantidad de valores '-1' en 'partnumber': {null_partnumber_count}")


Valores únicos en 'partnumber':
partnumber
-32760    1
 25473    1
 28822    1
 25251    1
 25252    1
         ..
 14798    1
 16734    1
 19097    1
 16511    1
 238      1
Name: count, Length: 43692, dtype: int64

Cantidad de valores '-1' en 'partnumber': 0


In [6]:
import numpy as np

# Generar recomendaciones para usuarios no logueados
def generate_unlogged_recommendations(test_df, products_df):
    """Genera recomendaciones diversificadas para usuarios no logueados en el formato correcto."""
    predictions = {}
    unlogged_sessions = test_df[test_df['user_id'] == -1]

    # Lista de productos más populares
    popular_products = products_df[
        (products_df['family'].isin(popular_families)) &
        (products_df['color_id'].isin(popular_colors))
    ]['partnumber'].drop_duplicates().tolist()

    for session_id in unlogged_sessions['session_id'].unique():
        # Selección aleatoria de 5 productos populares
        recommended_products = np.random.choice(popular_products, 5, replace=False).tolist()

        # Agregar al diccionario de predicciones
        predictions[str(session_id)] = recommended_products

    return {"target": predictions}

# Generar predicciones
unlogged_predictions = generate_unlogged_recommendations(test_df, products_df)

# Guardar predicciones en formato JSON
output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_unlogged_random.json"
with open(output_path, 'w') as f:
    json.dump(unlogged_predictions, f, indent=4)

print(f"Predicciones diversificadas para usuarios no logueados guardadas en {output_path}")


Predicciones diversificadas para usuarios no logueados guardadas en /home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_unlogged_random.json


---

# **Pipeline para usuarios logueados**

#### **Estrategia**
1. **Basado en historial**:
   - Utilizar las interacciones previas de cada usuario para generar recomendaciones personalizadas.
   - Usar atributos como `family`, `color_id`, y `similar_products` para recomendaciones basadas en contenido.

2. **Segmentación dinámica**:
   - Incorporar características como `RFM_score` y `user_class` para ajustar las recomendaciones.

3. **Modelo híbrido**:
   - Combinar enfoques colaborativos (similitud entre usuarios) y basados en contenido (atributos de productos).

### **Plan de acción**

1. **Preparar el historial de interacciones**:
   - Identificar los productos con los que cada usuario ha interactuado en el conjunto de entrenamiento.

2. **Entrenar un modelo colaborativo**:
   - Usar las interacciones previas para entrenar un modelo como `ALS` (Alternating Least Squares) o un modelo de factorización matricial.

3. **Incorporar atributos de contenido**:
   - Combinar atributos de usuario (`RFM_score`, `user_class`) y producto (`family`, `color_id`, `cluster`) para generar recomendaciones.

4. **Formato de salida**:
   - Generar un archivo JSON con las recomendaciones personalizadas para cada `session_id`.


In [1]:
from surprise import Dataset, Reader, SVD
import pandas as pd
import json

# Cargar datasets
train_df = pd.read_pickle("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_enriched.pkl")
test_df = pd.read_pickle("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl")
products_df = pd.read_pickle("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data_enriched.pkl")

# 1. Preparar datos para el modelo colaborativo
interactions = train_df[['user_id', 'partnumber', 'session_id']].copy()
interactions['rating'] = 1  # Asignar valor ficticio para indicar interacción
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(interactions[['user_id', 'partnumber', 'rating']], reader)

# Entrenar modelo colaborativo
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

# Filtrar valores negativos en 'partnumber'
products_df = products_df[products_df['partnumber'] > 0]


# Generar recomendaciones personalizadas para usuarios logueados
def generate_logged_recommendations(test_df, products_df, train_df, model):
    """Genera recomendaciones más personalizadas para usuarios logueados."""
    predictions = {}
    logged_sessions = test_df[test_df['user_id'] != -1]

    for _, row in logged_sessions.iterrows():
        user_id = row['user_id']
        session_id = row['session_id']

        # Obtener productos vistos por el usuario
        seen_products = train_df[train_df['user_id'] == user_id]['partnumber'].unique()

        # Filtrar productos no vistos
        unseen_products = products_df[~products_df['partnumber'].isin(seen_products)]['partnumber'].tolist()

        # Generar predicciones personalizadas
        recommended_products = []
        for product in unseen_products:
            pred = model.predict(user_id, product).est
            recommended_products.append((product, pred))

        # Ordenar por predicción y diversificar
        recommended_products = [int(p[0]) for p in sorted(recommended_products, key=lambda x: x[1], reverse=True)[:5]]

        # Agregar recomendaciones al diccionario
        predictions[str(session_id)] = recommended_products

    return {"target": predictions}

# Generar y guardar predicciones
logged_predictions = generate_logged_recommendations(test_df, products_df, train_df, model)

output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_logged.json"
with open(output_path, 'w') as f:
    json.dump(logged_predictions, f, indent=4)

print(f"Predicciones para usuarios logueados guardadas en {output_path}")


Predicciones para usuarios logueados guardadas en /home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_logged.json


---

# Combinar json

In [2]:
import json
import pandas as pd

# Rutas de los archivos
path_unlogged = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_unlogged.json"
path_logged = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_logged.json"
path_test_data = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl"
output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_final.json"

# Cargar archivos JSON existentes
with open(path_unlogged, 'r') as f:
    unlogged_predictions = json.load(f)["target"]

with open(path_logged, 'r') as f:
    logged_predictions = json.load(f)["target"]

# Cargar el dataset de prueba para obtener todos los session_id
test_df = pd.read_pickle(path_test_data)
all_session_ids = test_df['session_id'].unique()

# Combinar predicciones
final_predictions = {}
for session_id in all_session_ids:
    session_id_str = str(session_id)

    if session_id_str in logged_predictions:
        final_predictions[session_id_str] = logged_predictions[session_id_str]
    elif session_id_str in unlogged_predictions:
        final_predictions[session_id_str] = unlogged_predictions[session_id_str]
    else:
        # Si falta, llenar con productos más populares globalmente
        final_predictions[session_id_str] = [13211, 21402, 14933, 12765, 13402]  # Ejemplo

# Guardar el archivo final
with open(output_path, 'w') as f:
    json.dump({"target": final_predictions}, f, indent=4)

print(f"Archivo JSON final generado: {output_path}")


Archivo JSON final generado: /home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_final.json


: 